# Demonstration of transformer problems #

## Standard environment setup ##

In [4]:
import json

In [5]:
!python3 -m pip install folium==0.2.1

     |████████████████████████████████| 69 kB 6.0 MB/s             
  Preparing metadata (setup.py) ... done
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79809 sha256=8783dd8b57dbc54df4cf879858058ef8b9519b0d4f1388384bc8a25e2dfc6449
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


In [6]:
!python3 -m pip install imgaug==0.2.6

     |████████████████████████████████| 631 kB 13.8 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for imgaug: filename=imgaug-0.2.6-py3-none-any.whl size=654020 sha256=84d3c98a364d025f28f957762bcdee435f93db0928d5a45fe18a17dcf7097230
  Stored in directory: /root/.cache/pip/wheels/89/72/98/3ebfdba1069a9a8eaaa7ae7265cfd67d63ef0197aaee2e5f9c
Successfully built imgaug
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


In [7]:
!python3 -m pip install numpy==1.19

     |████████████████████████████████| 14.6 MB 12.7 MB/s            
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5


In [8]:
!git clone https://github.com/jaapkruijt/photobook-generation.git

Cloning into 'photobook-generation'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 144 (delta 61), reused 136 (delta 56), pack-reused 0
Receiving objects: 100% (144/144), 30.04 KiB | 10.01 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [9]:
!python3 -m pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple emissor

Looking in indexes: https://test.pypi.org/simple/, https://pypi.org/simple
     |████████████████████████████████| 345 kB 14.4 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 15.7 MB 57.2 MB/s            
     |████████████████████████████████| 47 kB 3.6 MB/s             
     |████████████████████████████████| 231 kB 50.4 MB/s            
     |████████████████████████████████| 130 kB 6.9 MB/s            
     |████████████████████████████████| 41 kB 580 kB/s             
  Created wheel for emissor: filename=emissor-0.0.dev5-py3-none-any.whl size=26584 sha256=bda1fe9e5a1f61c48303e4d0470cab1925670ec295856ce9df2d6fc29f78bba8
  Stored in directory: /root/.cache/pip/wheels/c8/3c/4e/6d3a8cca13deba8c5187f68142119f5aef2efee20e58d4d6a7
Successfully built emissor
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.0
    Uninstalling numpy-1.19.0:
      Successfully uninstalled numpy-1.19.0
ERROR: pip's dependency resolver

## Transformer problems exemplified ##

In [10]:
!pip install sentencepiece
!pip install transformers[sentencepiece]
!pip install --upgrade pip

## BART ##

In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_bart = AutoTokenizer.from_pretrained("eugenesiow/bart-paraphrase")

model_bart = AutoModelForSeq2SeqLM.from_pretrained("eugenesiow/bart-paraphrase")

In [12]:
pic1 = json.load(open('./photobook-generation/data/scenario_1/picture_1/picture_1.json'))

In [13]:
persons_list = pic1['context']['persons']

In [14]:
person_dict = persons_list[0]

In [15]:
desc = '. '.join([k + ': ' + ', '.join(v) for k, v in person_dict.items()]).replace('_', ' ')

In [16]:
def paraphrase_bart(tokenizer, model, sentence):
  return tokenizer.batch_decode(model.generate(tokenizer(sentence, return_tensors='pt')['input_ids']), skip_special_tokens=True)

In [17]:
desc

'name: Emily. full name: Emily Waltham. spouse: Ross Geller. sibling: Emma Waltham. sex or gender: female. country of citizenship: United Kingdom'

In [18]:
paraphrase_bart(tokenizer_bart, model_bart, desc)

['Name: Emily Waltham. Full name: Ross Geller. sibling: Emma W']

In [19]:
artificial_sentences = "Emily Waltham's spouse is Ross Geller. Emily Waltham's sibling is Emma Waltham. Emily Waltham's sex or gender is female. Emily Waltham's country of citizenship is United Kingdom."

In [20]:
paraphrase_bart(tokenizer_bart, model_bart, artificial_sentences)

["The spouse of Emily Waltham is Ross Geller, Emily WALTHam's sibling"]

In [21]:
artificial_sentences2 = "Emily Waltham's spouse is Ross Geller. Emily Waltham's sibling is Emma Waltham. Emily Waltham's country of citizenship is United Kingdom. Emily Waltham's sex or gender is female. "

In [22]:
paraphrase_bart(tokenizer_bart, model_bart, artificial_sentences2)

["The spouse of Emily Waltham is Ross Geller, Emily WALTHam's sibling"]

In [23]:
artificial_sentences3 = "Emily Waltham's spouse is Ross Geller. Emily Waltham's country of citizenship is United Kingdom. Emily Waltham's sex or gender is female. Emily Waltham's sibling is Emma Waltham. "

In [24]:
paraphrase_bart(tokenizer_bart, model_bart, artificial_sentences3)

["The spouse of Emily Waltham is Ross Geller. Emily WALTHam's country"]

## T5 ##

In [25]:
tokenizer_t5 = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")

model_t5 = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws")

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [26]:
encoding_desc = tokenizer_t5.encode_plus("paraphrase: " + desc, padding=True, return_tensors='pt')

In [27]:
encoding_desc

{'input_ids': tensor([[ 3856, 27111,    10,   564,    10, 15818,     5,   423,   564,    10,
         15818, 14591,  1483,     5,  9911,    10,  9616,   350,  9670,     5,
           108,  7428,    10, 15325, 14591,  1483,     5,     3,     7,   994,
            42,  7285,    10,  3955,     5,   684,    13, 22214,    10,   907,
          6524,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [28]:
outputs_desc = model_t5.generate(
    input_ids=encoding_desc['input_ids'], attention_mask=encoding_desc['attention_mask'],
    max_length=256,
    do_sample=True,
    top_k=120,
    top_p=0.95,
    early_stopping=True,
    num_return_sequences=5
)

In [29]:
for output in outputs_desc:
  print(tokenizer_t5.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True))

Emmy. full name: Emily Waltham. spouse: Ross Geller. sibling: Emma Waltham. sex or gender: male. country of citizenship: UK
sex or gender: female. Country of citizenship: United Kingdom
Full name: Emily Waltham. Spouse: Ross Geller. Silent: Emma Waltham. sex or gender: female. country of citizenship:
Ehebesicsname: Emily. full name: Emily Waltham. spouse: Ross Geller. sexist or gender: female. country of citizenship: United Kingdom
Name: Emily. Full name: Emily Waltham. husband: Ross Geller. sibling: Emma Waltham. sex or gender: female. country of citizenship: United Kingdom


In [30]:
def paraphrase_t5(tokenizer, model, sentence, max_length=32):
  encoding = tokenizer.encode_plus("paraphrase: " + sentence, padding=True, return_tensors='pt')
  outputs = model_t5.generate(
    input_ids=encoding['input_ids'], attention_mask=encoding['attention_mask'],
    max_length=max_length, do_sample=True, top_k=120, top_p=0.95,early_stopping=True,
    num_return_sequences=5)
  
  return [tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True) for output in outputs]

In [31]:
paraphrase_t5(tokenizer_t5, model_t5, desc)

['Full name: Emily Waltham. spouse: Ross Geller. sex or gender: female. country of citizenship: United Kingdom',
 'Full name: Emily Waltham. Spouse: Ross Geller. Singing Sibling: Emma Waltham. sex or gender: female',
 'Full name: Emily. spouse: Ross Geller. sibling: Emma Waltham. sex or gender: female. country of citizenship:',
 'Name: Emily. Full name: Emily Waltham. spouse: Ross Geller. sibling: Emma Waltham. sex or gender: female',
 'Name: Emily. Full name: Emily Waltham. Spouse: Ross Geller. Simplicity: Emma Waltham. sex or gender']

In [32]:
paraphrase_t5(tokenizer_t5, model_t5, artificial_sentences)

["Emily Waltham's spouse is Ross Geller. Emily Waltham's sister is Emma Waltham. Emily Waltham's sex is",
 "Emily Waltham's spouse is Ross Geller, Emily Waltham's sister is Emma Waltham. Emily Waltham sex or gender",
 "Emily Waltham's spouse is Ross Geller, Emily Waltham's sibling is Emma Waltham. Emily Waltham is sex",
 "Emily Waltham's spouse is Ross Geller, Emily Waltham's sibling is Emma Waltham, Emily Waltham's ",
 "Emily Waltham's spouse is Ross Geller, Emily Waltham's sibling is Emma Waltham, Emily Waltham sex"]

In [33]:
paraphrase_t5(tokenizer_t5, model_t5, artificial_sentences, max_length=100)

["Emily Waltham's spouse is Ross Geller. Emily Waltham’ s sister is Emma Waltham. Emily Waltham’ s sex or gender is female and Emily Waltham s citizenship country is United Kingdom.",
 "Emily Waltham's spouse is Ross Geller, Emily Waltham's sibling is Emma Waltham, Emily Waltham's sexual or gender is female. Emily Waltham's country of birth is United Kingdom.",
 "Emily Waltham is Ross Geller; Emily Waltham's sibling is Emma Waltham ; Emily Waltham's sex or gender is female ; Emily Waltham's country of citizenship is the United Kingdom.",
 'Emily Waltham’s spouse is Ross Geller and Emily Waltham’s sibling is Emma Waltham. Emily Waltham’s sex or gender is female. Emily Waltham’s citizenship is United Kingdom.',
 "Emily Waltham's spouse is Ross Geller and Emily Waltham's sibling is Emma Waltham. Emily Waltham's sex or gender is female and Emily Waltham's country of citizenship is the United Kingdom."]

In [34]:
paraphrase_t5(tokenizer_t5, model_t5, artificial_sentences2)

["Emily Waltham's spouse is Ross Geller, Emily Waltham's sibling is Emma Waltham. Emily Waltham's citizenship country",
 "Emily Waltham's spouse is Ross Geller and Emily Waltham's younger sister is Emma Waltham. Emily Waltham's country of citizenship",
 "Emily Waltham's spouse is Ross Geller, Emily Waltham's sibling is Emma Waltham. Emily Waltham's",
 "Emily Waltham's spouse is Ross Geller ; Emily Waltham's sibling is Emma Waltham ; Emily Waltham's",
 "Emily Waltham's spouse is Ross Geller, Emily Waltham's sibling is Emma Waltham, Emily Waltham’ s"]

In [35]:
paraphrase_t5(tokenizer_t5, model_t5, artificial_sentences3)

["Emily Waltham is Ross Geller, Emily Waltham's country of residence is United Kingdom, Emily Waltham's sex or",
 "Emily Waltham's spouse is Ross Geller, Emily Waltham's country of citizenship is the United Kingdom, Emily Waltham's",
 "Emily Waltham's spouse is Ross Geller, Emily Waltham's country of citizenship is United Kingdom, Emily Waltham's ",
 "Emily Waltham's spouse is Ross Geller. Emily Waltham is of United Kingdom. Emily Waltham is male. Emily Waltham",
 "Emily Waltham's spouse is Ross Geller, Emily Waltham's country of citizenship is the United Kingdom, Emily Waltham's s"]

## GPT-2 ##

In [36]:
from transformers import pipeline
generator = pipeline('text-generation', model='gpt2')

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

## Either does nothing ##

In [44]:
generator(artificial_sentences, max_length=30, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 46, but ``max_length`` is set to 30. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


[{'generated_text': "Emily Waltham's spouse is Ross Geller. Emily Waltham's sibling is Emma Waltham. Emily Waltham's sex or gender is female. Emily Waltham's country of citizenship is United Kingdom.\n"},
 {'generated_text': "Emily Waltham's spouse is Ross Geller. Emily Waltham's sibling is Emma Waltham. Emily Waltham's sex or gender is female. Emily Waltham's country of citizenship is United Kingdom. Emily"},
 {'generated_text': "Emily Waltham's spouse is Ross Geller. Emily Waltham's sibling is Emma Waltham. Emily Waltham's sex or gender is female. Emily Waltham's country of citizenship is United Kingdom.\n"},
 {'generated_text': "Emily Waltham's spouse is Ross Geller. Emily Waltham's sibling is Emma Waltham. Emily Waltham's sex or gender is female. Emily Waltham's country of citizenship is United Kingdom. Emily"},
 {'generated_text': "Emily Waltham's spouse is Ross Geller. Emily Waltham's sibling is Emma Waltham. Emily Waltham's sex or gender is female. Emily Waltham's country of cit

In [46]:
str(person_dict)

"{'name': ['Emily'], 'full_name': ['Emily Waltham'], 'spouse': ['Ross Geller'], 'sibling': ['Emma Waltham'], 'sex_or_gender': ['female'], 'country_of_citizenship': ['United Kingdom']}"

In [47]:
generator(str(person_dict), max_length=30, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 66, but ``max_length`` is set to 30. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


[{'generated_text': "{'name': ['Emily'], 'full_name': ['Emily Waltham'], 'spouse': ['Ross Geller'], 'sibling': ['Emma Waltham'], 'sex_or_gender': ['female'], 'country_of_citizenship': ['United Kingdom']}]"},
 {'generated_text': "{'name': ['Emily'], 'full_name': ['Emily Waltham'], 'spouse': ['Ross Geller'], 'sibling': ['Emma Waltham'], 'sex_or_gender': ['female'], 'country_of_citizenship': ['United Kingdom']}\n"},
 {'generated_text': "{'name': ['Emily'], 'full_name': ['Emily Waltham'], 'spouse': ['Ross Geller'], 'sibling': ['Emma Waltham'], 'sex_or_gender': ['female'], 'country_of_citizenship': ['United Kingdom']} #"},
 {'generated_text': "{'name': ['Emily'], 'full_name': ['Emily Waltham'], 'spouse': ['Ross Geller'], 'sibling': ['Emma Waltham'], 'sex_or_gender': ['female'], 'country_of_citizenship': ['United Kingdom']}\n"},
 {'generated_text': "{'name': ['Emily'], 'full_name': ['Emily Waltham'], 'spouse': ['Ross Geller'], 'sibling': ['Emma Waltham'], 'sex_or_gender': ['female'], 'countr

## Or hallucinates nonsense ##

In [41]:
' '.join(artificial_sentences.split()[:-17])

"Emily Waltham's spouse is Ross Geller. Emily Waltham's sibling is"

In [42]:
generator(' '.join(artificial_sentences.split()[:-17]), max_length=30, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Emily Waltham's spouse is Ross Geller. Emily Waltham's sibling is James.\n\nThere are about 10.3 million U"},
 {'generated_text': "Emily Waltham's spouse is Ross Geller. Emily Waltham's sibling is Joseph and her sister is Rebecca.\n\nAdvertisement\n\n"},
 {'generated_text': "Emily Waltham's spouse is Ross Geller. Emily Waltham's sibling is Nick Geller.\n\nThere has never been any such"},
 {'generated_text': "Emily Waltham's spouse is Ross Geller. Emily Waltham's sibling is Robert Geller. Both parents are divorced and live in Chicago"},
 {'generated_text': "Emily Waltham's spouse is Ross Geller. Emily Waltham's sibling is Rachel S. Sibbins & Rachel Spencer is daughter"}]